In [1]:
import psutil
import matplotlib.pyplot as plt
import pandas as pd
import time
import csv
import random
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go


In [78]:
# 기존 CSV 파일 경로
input_file = "user2_battery_data.csv"
output_file = "user2_processed_data.csv"

# 데이터를 읽어서 전처리 수행
processed_data = []
with open(input_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    prev_battery_percentage = None

    for row in reader:
        # 시간 파싱
        current_time = datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S")
        current_battery_percentage = float(row["Current battery percentage"])
        
        # 배터리 사용량 계산
        battery_usage = None
        if prev_battery_percentage is not None:
            battery_usage = round(prev_battery_percentage - current_battery_percentage, 2)
        
        # 요일 추가 및 배터리 사용량 기록
        processed_data.append({
            "time": row["time"],
            "weekday": current_time.weekday(),
            "CPU utilization percentage": row["CPU utilization percentage"],
            "GPU Utilization": row["GPU Utilization"],
            "Current battery percentage": row["Current battery percentage"],
            "Battery Status": row["Battery Status"],
            "Battery Usage": battery_usage
        })
        
        prev_battery_percentage = current_battery_percentage

# 전처리된 데이터를 새로운 CSV로 저장
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    fieldnames = [
        "time", "weekday", "CPU utilization percentage", "GPU Utilization",
        "Current battery percentage", "Battery Status", "Battery Usage"
    ]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(processed_data)

print(f"Processed data has been saved to {output_file}.")

Processed data has been saved to user2_processed_data.csv.


In [79]:
# Read CSV for visualization
df = pd.read_csv("user2_processed_data.csv")

# Plot the data for visualization using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time"], y=df["Battery Usage"],
                         mode='lines', name='Battery Usage',
                         line=dict(color='blue')))

fig.update_layout(
    title='Battery Usage Over Time',
    xaxis_title='Time',
    yaxis_title='Battery Usage',
    xaxis_rangeslider_visible=True
)

fig.show()

In [80]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, GRU, Dense, LSTM, TimeDistributed
from keras.optimizers.legacy import SGD
from keras.optimizers.schedules import ExponentialDecay

def LSTM_model(x_train, y_train):
    model = Sequential()

    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1), activation='tanh'))
    model.add(LSTM(units=50, return_sequences=True, activation='tanh'))
    model.add(TimeDistributed(Dense(units=1)))  # 각 시간 단계마다 독립적인 예측

    # Compile
    model.compile(optimizer=SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=False),
                  loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=20, batch_size=32, verbose=1)


    return model

def GRU_model(x_train, y_train):

    # 모델 정의
    model = Sequential()

    # GRU 레이어 추가
    model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1), activation='tanh'))
    model.add(GRU(units=50, return_sequences=True, activation='tanh'))

    # 각 타임스텝별 Dense 레이어
    model.add(TimeDistributed(Dense(units=1)))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 모델 학습
    model.fit(x_train, y_train, epochs=20, batch_size=32, verbose=1)

    return model


In [93]:
from sklearn.preprocessing import MinMaxScaler

# CSV 데이터 읽기
data = pd.read_csv("user2_processed_data.csv")

# 'time' 컬럼을 datetime 형식으로 변환
data['time'] = pd.to_datetime(data['time'])

# 필요한 변수 초기화
time_steps = 144  # 입력 데이터 시퀀스 길이 (예: 하루)
for_periods = 144  # 예측할 시퀀스 길이 (예: 하루)

# Train/Test 분리
start_date = data['time'].iloc[1]
end_date = data['time'].iloc[len(data) - time_steps+17]
mask2_date = data['time'].iloc[len(data) - (time_steps+127)]
print(f"mask2_date: {mask2_date}")
print(f"start_date: {start_date}")
print(f"end_data: {end_date}")
mask1 = (data['time'] >= start_date) & (data['time'] <= end_date)
mask2 = (data['time'] >= mask2_date) & (data['time'] <= end_date)

ts_train = data.loc[mask1]
ts_test = data.loc[mask2]

# MinMaxScaler로 데이터 정규화
sc = MinMaxScaler(feature_range=(0, 1))

ts_train_scaled = sc.fit_transform(ts_train['Current battery percentage'].values.reshape(-1, 1))
ts_test_scaled = sc.transform(ts_test['Current battery percentage'].values.reshape(-1, 1))

# x_train, y_train 만들기
x_train = []
y_train = []

for i in range(time_steps, len(ts_train_scaled) - for_periods):
    x_train.append(ts_train_scaled[i-time_steps:i, 0])
    y_train.append(ts_train_scaled[i:i+for_periods, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# RNN/LSTM 입력 형태로 reshape
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

# x_test 만들기
inputs = np.concatenate((ts_train_scaled[-time_steps:], ts_test_scaled[:for_periods]))
x_test = []

for i in range(time_steps, len(inputs) - for_periods + 1):
    x_test.append(inputs[i-time_steps:i])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")


mask2_date: 2024-12-15 23:50:00
start_date: 2024-11-16 00:10:00
end_data: 2024-12-16 23:50:00
x_train shape: (4175, 144, 1)
y_train shape: (4175, 144, 1)
x_test shape: (1, 144, 1)


In [94]:
print("x_train shape:", x_train.shape)  # (샘플 수, time_steps, 특성 수)
print("y_train shape:", y_train.shape)  # (샘플 수, for_periods, 특성 수)


model = LSTM_model(x_train, y_train)
#model = GRU_model(x_train, y_train)
model.summary()

model.save("user2_model.h5")


x_train shape: (4175, 144, 1)
y_train shape: (4175, 144, 1)
Epoch 1/20
131/131 [==============================] - 3s 16ms/step - loss: 0.0730
Epoch 2/20
131/131 [==============================] - 2s 15ms/step - loss: 0.0527
Epoch 3/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0524
Epoch 4/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0523
Epoch 5/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0521
Epoch 6/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0521
Epoch 7/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0521
Epoch 8/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0520
Epoch 9/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0520
Epoch 10/20
131/131 [==============================] - 2s 16ms/step - loss: 0.0519
Epoch 11/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0520
Epoch 12/20
131/131 [===============

/home/lmls/jisoo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [95]:
weights = model.get_weights()
print(weights)
print(len(weights))
print("x_test shape:", x_test.shape)
lstm_prediction = model.predict(x_test)
prediction_2d = lstm_prediction.reshape(-1, 1)

[array([[-4.36774008e-02,  1.41840115e-01,  1.44122779e-01,
        -3.02684810e-02, -1.03381954e-01, -1.43605992e-01,
         1.18191317e-01,  1.13457479e-01, -1.73181787e-01,
         1.74213052e-01,  1.19991750e-01, -1.66395813e-01,
        -5.50138950e-02,  4.20622155e-02, -3.07028815e-02,
        -1.23736501e-01, -1.16966918e-01,  7.41043538e-02,
        -8.18060637e-02,  4.82753589e-04, -1.16089396e-01,
        -1.03660166e-01, -4.22519334e-02, -7.43456036e-02,
        -3.64842191e-02,  1.67730376e-01, -7.96702951e-02,
         1.19912989e-01, -7.21255168e-02, -1.10896915e-01,
        -9.93319303e-02, -1.48651332e-01, -4.70215380e-02,
         2.28492394e-02, -4.78141867e-02,  1.66180789e-01,
         1.43155977e-01,  8.14738646e-02, -1.85417626e-02,
         1.07928634e-01, -1.86919072e-03,  1.32222518e-01,
         7.58555904e-02,  3.33157480e-02,  9.57730636e-02,
        -1.29980713e-01, -1.02345377e-01,  2.75595915e-02,
        -1.47234976e-01,  7.63367340e-02,  1.48149446e-

In [85]:
weights = model.get_weights()
print(weights)
print(len(weights))
print("x_test shape:", x_test.shape)
gru_prediction = model.predict(x_test)
prediction_2d = gru_prediction.reshape(-1, 1)

[array([[ 0.44908777, -0.00160819,  0.12130648,  0.04080487, -0.21856509,
        -0.33107853,  0.17272793,  0.20443389, -0.16777328,  0.18338864,
        -0.13878134, -0.02385298,  0.11847317,  0.01983008, -0.03882223,
         0.02366397,  0.10878067, -0.03884645, -0.24818419,  0.04803518,
        -0.03225313, -0.07849512, -0.22291788,  0.1552603 ,  0.11384622,
        -0.12956457, -0.2611591 , -0.16882029,  0.5016013 ,  0.08926188,
        -0.07864022, -0.11948633,  0.15526932,  0.02648814,  0.00215742,
         0.16936128, -0.04176382, -0.07327741,  0.00457246,  0.6093925 ,
        -0.266712  , -0.38336605,  0.06169008,  0.59239346, -0.09462667,
        -0.10955524, -0.09227883,  0.10211352,  0.1299254 , -0.4014587 ,
         0.22558117,  0.07409484,  0.42169875,  0.13377209, -0.29724774,
        -0.10675311, -0.01247215, -0.09331805, -0.07273942,  0.14754377,
        -0.13159394, -0.02000909, -0.18524331, -0.11803342, -0.03281866,
        -0.07875961,  0.07602654,  0.06179892,  0.

In [96]:
predicted_dates = pd.date_range(start=end_date, periods=for_periods, freq='10T')
print(predicted_dates)
lstm_predict = lstm_prediction
lstm_predict = lstm_predict * 100
print(lstm_predict)


DatetimeIndex(['2024-12-16 23:50:00', '2024-12-17 00:00:00',
               '2024-12-17 00:10:00', '2024-12-17 00:20:00',
               '2024-12-17 00:30:00', '2024-12-17 00:40:00',
               '2024-12-17 00:50:00', '2024-12-17 01:00:00',
               '2024-12-17 01:10:00', '2024-12-17 01:20:00',
               ...
               '2024-12-17 22:10:00', '2024-12-17 22:20:00',
               '2024-12-17 22:30:00', '2024-12-17 22:40:00',
               '2024-12-17 22:50:00', '2024-12-17 23:00:00',
               '2024-12-17 23:10:00', '2024-12-17 23:20:00',
               '2024-12-17 23:30:00', '2024-12-17 23:40:00'],
              dtype='datetime64[ns]', length=144, freq='10min')
[[[60.970585]
  [65.38124 ]
  [68.06582 ]
  [69.54869 ]
  [70.228874]
  [70.39884 ]
  [70.25399 ]
  [69.95834 ]
  [69.60153 ]
  [69.251656]
  [68.98532 ]
  [68.73154 ]
  [68.53886 ]
  [68.41147 ]
  [68.28333 ]
  [68.154   ]
  [68.088844]
  [67.99518 ]
  [67.92254 ]
  [67.85154 ]
  [67.818115]
  [67.79811 

/tmp/ipykernel_6763/3265961440.py:1: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



In [88]:
predicted_dates = pd.date_range(start=end_date, periods=for_periods, freq='10T')
print(predicted_dates)
gru_predict = gru_prediction
gru_predict = gru_predict * 100
gru_predict = np.clip(gru_predict, 0, 100)
print(gru_predict)



DatetimeIndex(['2024-12-16 23:50:00', '2024-12-17 00:00:00',
               '2024-12-17 00:10:00', '2024-12-17 00:20:00',
               '2024-12-17 00:30:00', '2024-12-17 00:40:00',
               '2024-12-17 00:50:00', '2024-12-17 01:00:00',
               '2024-12-17 01:10:00', '2024-12-17 01:20:00',
               ...
               '2024-12-17 22:10:00', '2024-12-17 22:20:00',
               '2024-12-17 22:30:00', '2024-12-17 22:40:00',
               '2024-12-17 22:50:00', '2024-12-17 23:00:00',
               '2024-12-17 23:10:00', '2024-12-17 23:20:00',
               '2024-12-17 23:30:00', '2024-12-17 23:40:00'],
              dtype='datetime64[ns]', length=144, freq='10min')
[[[67.174   ]
  [68.544014]
  [68.227844]
  [68.91312 ]
  [68.75936 ]
  [68.59612 ]
  [68.56665 ]
  [68.29953 ]
  [68.32949 ]
  [68.27317 ]
  [67.988914]
  [68.768654]
  [68.30575 ]
  [68.30494 ]
  [68.782776]
  [68.620476]
  [68.11683 ]
  [68.8246  ]
  [68.382416]
  [68.53529 ]
  [68.29921 ]
  [68.525276

/tmp/ipykernel_6763/4233508430.py:1: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



In [89]:

# 그래프 객체 생성
fig = go.Figure()

# 기존 전력 사용량 데이터 추가
fig.add_trace(go.Scatter(x=data['time'], y=data['Current battery percentage'],
                         mode='lines', name='Actual battery percentage'))

# 예측 전력 사용량 데이터 추가 (여기서 수정된 부분)
fig.add_trace(go.Scatter(x=predicted_dates, y=gru_predict.flatten(),
                         mode='lines', name='Predicted battery percentage',
                         line=dict(color='tomato')))  # 예측 데이터에는 다른 색상을 사용

# 레이아웃 업데이트
fig.update_layout(
    title='GRU battery percentage',
    xaxis_title='Time',
    yaxis_title='Current battery percentage',
    xaxis_rangeslider_visible=True
)

# 그래프 표시
fig.show()

In [97]:

# 그래프 객체 생성
fig = go.Figure()

# 기존 전력 사용량 데이터 추가
fig.add_trace(go.Scatter(x=data['time'], y=data['Current battery percentage'],
                         mode='lines', name='Actual battery percentage'))

# 예측 전력 사용량 데이터 추가 (여기서 수정된 부분)
fig.add_trace(go.Scatter(x=predicted_dates, y=lstm_predict.flatten(),
                         mode='lines', name='Predicted battery percentage',
                         line=dict(color='tomato')))  # 예측 데이터에는 다른 색상을 사용

# 레이아웃 업데이트
fig.update_layout(
    title='GRU battery percentage',
    xaxis_title='Time',
    yaxis_title='Current battery percentage',
    xaxis_rangeslider_visible=True
)

# 그래프 표시
fig.show()

In [90]:
print(f"Length of predicted_dates: {len(predicted_dates)}")
print(f"Length of gru_predict: {len(gru_predict)}")


Length of predicted_dates: 144
Length of gru_predict: 1


In [91]:
import pandas as pd

# 기존 CSV 파일 경로
input_file = "user2_processed_data.csv"
output_file = "user2_predictions.csv"

# 기존 데이터를 읽어오기
data = pd.read_csv(input_file)

# 예측 데이터와 날짜 생성
predicted_dates = pd.date_range(start=end_date, periods=for_periods, freq="10T")
gru_predict = gru_prediction.squeeze() * 100  # 1차원으로 변환하고 값 조정

# 기존 데이터에 예측값을 추가할 DataFrame 생성
predicted_df = pd.DataFrame({
    "time": predicted_dates,
    "prediction": gru_predict
})

# 기존 데이터의 'time' 컬럼을 datetime 형식으로 변환
data["time"] = pd.to_datetime(data["time"])

# 예측값과 기존 데이터를 병합
final_data = pd.merge(data, predicted_df, on="time", how="outer")

# 최종 데이터를 새로운 CSV 파일로 저장
final_data.to_csv(output_file, index=False, encoding="utf-8")

print(f"Final data with predictions has been saved to {output_file}.")


Final data with predictions has been saved to user2_predictions.csv.


/tmp/ipykernel_6763/4099535942.py:11: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



In [48]:
from keras.models import Sequential, load_model

local_model = load_model("global_model.h5")

local_model.fit(x_train, y_train, epochs=20, batch_size=32, verbose=1)


Epoch 1/20
126/126 [==============================] - 2s 10ms/step - loss: 0.0599
Epoch 2/20
126/126 [==============================] - 2s 13ms/step - loss: 0.0471
Epoch 3/20
126/126 [==============================] - 2s 14ms/step - loss: 0.0465
Epoch 4/20
126/126 [==============================] - 2s 13ms/step - loss: 0.0454
Epoch 5/20
126/126 [==============================] - 2s 14ms/step - loss: 0.0433
Epoch 6/20
126/126 [==============================] - 2s 13ms/step - loss: 0.0426
Epoch 7/20
126/126 [==============================] - 2s 13ms/step - loss: 0.0422
Epoch 8/20
126/126 [==============================] - 2s 15ms/step - loss: 0.0420
Epoch 9/20
126/126 [==============================] - 2s 14ms/step - loss: 0.0419
Epoch 10/20
126/126 [==============================] - 1s 12ms/step - loss: 0.0415
Epoch 11/20
126/126 [==============================] - 1s 11ms/step - loss: 0.0414
Epoch 12/20
126/126 [==============================] - 2s 15ms/step - loss: 0.0413
Epoch 13/20
1

In [50]:
weights = model.get_weights()
print(weights)
print(len(weights))
print("x_test shape:", x_test.shape)
gru_prediction = model.predict(x_test)
prediction_2d = gru_prediction.reshape(-1, 1)
predicted_dates = pd.date_range(start=end_date, periods=for_periods, freq='10T')
print(predicted_dates)
gru_predict = gru_prediction
gru_predict = gru_predict * 100
gru_predict = np.clip(gru_predict, 0, 100)
print(gru_predict)

[array([[-0.27555272, -0.31061706, -0.13207667, -0.38714907, -0.04175662,
         0.05239299, -0.2949016 , -0.36209399, -0.09780579,  0.00209423,
        -0.17491585, -1.066949  ,  0.18482158, -0.23563626, -0.06181383,
         0.39187014,  0.09627142, -0.5128304 , -0.1829159 , -0.30908215,
         0.00477561, -0.51427823, -0.06539645, -0.33673105, -0.35092667,
        -0.3202057 , -0.10280006, -0.25660044, -0.29741913, -0.07874983,
        -0.01921112,  0.06195594, -0.24665141, -0.07313079,  0.00265197,
        -0.06749033,  0.04831576, -0.41015187, -1.2416004 , -0.08980484,
        -0.10243341, -0.27966374, -0.812626  , -0.38838932, -0.1452872 ,
        -0.17246643, -0.5296961 , -0.7656908 , -0.20411058, -0.29070038,
        -0.09421412, -0.27693787, -0.18516721, -0.03681872,  0.11413456,
        -0.06366824, -0.11136711, -0.08584841, -0.07505436, -0.0028052 ,
         0.09228495,  0.16414429, -0.39638475,  0.13627987, -0.24175458,
         0.09863027,  0.1771248 , -0.16362777, -0.

/tmp/ipykernel_2575585/2359208918.py:7: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



In [51]:

# 그래프 객체 생성
fig = go.Figure()

# 기존 전력 사용량 데이터 추가
fig.add_trace(go.Scatter(x=data['time'], y=data['Current battery percentage'],
                         mode='lines', name='Actual battery percentage'))

# 예측 전력 사용량 데이터 추가 (여기서 수정된 부분)
fig.add_trace(go.Scatter(x=predicted_dates, y=gru_predict.flatten(),
                         mode='lines', name='Predicted battery percentage',
                         line=dict(color='tomato')))  # 예측 데이터에는 다른 색상을 사용

# 레이아웃 업데이트
fig.update_layout(
    title='GRU battery percentage',
    xaxis_title='Time',
    yaxis_title='Current battery percentage',
    xaxis_rangeslider_visible=True
)

# 그래프 표시
fig.show()